In [297]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stock_module as sm
import random
import os
import datetime
import time
import datetime
from datetime import date

In [318]:
def book_values(stock_name: str):
    """
    AMAÇ: Belirtilen hisse senedi için, her mali dönemin defter değerini (özkaynak) getirmek
    """
    bv = []
    for term in [3,6,9,12]:
        bv.append(sm.get_financial_ratio([stock_name], term, "equity"))
    bv = pd.concat(bv)
    
    new = pd.DataFrame(bv.index.str.split('/',1).tolist(), columns = ['year','month'])
    bv = bv.reset_index(drop=True)
    bv = bv.join(new)
    bv['day'] = np.where(bv['month']=="3", 31, (np.where(bv['month']=="12", 31, 30)))
    
    bv["year"] = bv["year"].apply(int)
    bv["month"] = bv["month"].apply(int)
    df = pd.DataFrame(pd.to_datetime(pd.DataFrame({'year': bv["year"], 'month': bv["month"], 'day': bv["day"]})))
    bv = bv.join(df)
    
    bv["date"] = bv[0]
    bv = bv.drop(['year', 'month', "day", 0], axis=1)
    bv.index = bv["date"]
    bv["book_value"] = bv.iloc[:,0]
    bv = pd.DataFrame(bv.iloc[:,2])
    
    bv = bv.sort_index()
    bv["stock_name"] = stock_name
    bv = bv.reset_index()
    
    return bv

In [294]:
frame = book_values("THYAO")

In [308]:
def day_table(frame_book_values):
    """
    Her mali dönemden bir sonraki mali döneme kadar kaç gün var.
    """
    fram = pd.DataFrame(frame_book_values["date"].diff())
    fram.columns = ["days"]
    fram = pd.DataFrame(fram["days"].shift(periods=-1))
    fram["date"] = frame_book_values["date"]
    fram.iloc[-1,0] = date.today() - pd.DataFrame(fram["date"]).iloc[-1,:][0].date()
    fram["days"] = pd.DataFrame(fram["days"].dt.days)
    
    return fram

In [309]:
days = day_table(frame)

In [337]:
def book_value_timeFrame(stock_name:str, frame_book_values, days):
    """
    AMAÇ: Belirtilen hisse senedinin defter değerini zaman serisi olarak oluşturmak.
    """
    frames = []
    for day, date, seq in zip(days["days"], days["date"], range(frame_book_values.shape[0])):
        ind = pd.date_range(date, periods = day, freq ='1d')
        df = pd.DataFrame(index = ind) 
        df["book_value"] = frame_book_values.iloc[seq,:][1]
        df["stock_name"] = frame_book_values.iloc[seq,:][2]
        frames.append(df)
    frames = pd.concat(frames)
    
    return frames

In [341]:
stock_name = "THYAO"
frame_book_values = book_values(stock_name)
days = day_table(frame_book_values)

In [343]:
book_value_timeFrame(stock_name, frame_book_values, days)

,book_value,stock_name
2008-03-31,2.116202e+09,THYAO
2008-04-01,2.116202e+09,THYAO
2008-04-02,2.116202e+09,THYAO
2008-04-03,2.116202e+09,THYAO
2008-04-04,2.116202e+09,THYAO
...,...,...
2020-06-07,4.134300e+10,THYAO
2020-06-08,4.134300e+10,THYAO
2020-06-09,4.134300e+10,THYAO
2020-06-10,4.134300e+10,THYAO


In [380]:
def book_valueFrame(stock_name:str, frame):
    
    stock_values = sm.clean_stock_raw_table(stock_name)
    stock_values = stock_values[["HGDG_HS_KODU", "HGDG_TARIH", "PD"]]
    stock_values.index = stock_values.HGDG_TARIH
    stock_values.index.names = ['Date']
    del stock_values["HGDG_TARIH"]
    
    bookValues = frame.join(stock_values)
    bookValues = bookValues.dropna()
    
    bookValues = bookValues.rename(columns={"PD": "market_value"})
    bookValues["marketV_to_bookV"] = bookValues["market_value"] / bookValues["book_value"]
    
    del bookValues["HGDG_HS_KODU"]
    
    return bookValues

In [381]:
stock_name = "BIMAS"

In [382]:
frame_book_values = book_values(stock_name)
days = day_table(frame_book_values)

In [383]:
frame = book_value_timeFrame(stock_name, frame_book_values, days)

In [384]:
book_valueFrame(stock_name, frame)

,book_value,stock_name,market_value,marketV_to_bookV
2008-03-31,2.351410e+08,BIMAS,2.656500e+09,11.297477
2008-04-01,2.351410e+08,BIMAS,2.631200e+09,11.189882
2008-04-02,2.351410e+08,BIMAS,2.605900e+09,11.082287
2008-04-03,2.351410e+08,BIMAS,2.504700e+09,10.651907
2008-04-04,2.351410e+08,BIMAS,2.530000e+09,10.759502
...,...,...,...,...
2020-06-02,4.678679e+09,BIMAS,3.828396e+10,8.182643
2020-06-03,4.678679e+09,BIMAS,3.968052e+10,8.481137
2020-06-04,4.678679e+09,BIMAS,4.189680e+10,8.954835
2020-06-05,4.678679e+09,BIMAS,4.031808e+10,8.617407


In [397]:
def update_market_value_frame():
    frames = []
    stockNames = sm.get_stock_names()
    for stock in stockNames:
        stock_name = stock
        try:
            frame_book_values = book_values(stock_name)
            days = day_table(frame_book_values)
            frame = book_value_timeFrame(stock_name, frame_book_values, days)
            final = book_valueFrame(stock_name, frame)
            frames.append(final)
        except:
            print(stock_name)
    frames = pd.concat(frames)
    
    return frames

In [399]:
marketValues = update_market_value_frame()

AKGRT
VAKBN
ISBTR
BAYRK
QNBFB
SEKFK
ISATR
ISFIN
OYAKC
ISKUR
PAPIL
ISCTR
ARDYZ
ANSGR
CRDFA
GARFA
AVISA
YKBNK
ALBRK
HALKS
GUSGR
VAKFN
KLNMA
RAYSG
QNBFL
ANHYT
DOCO
GARAN
HALKB
SKBNK
ICBCT
LIDFA
TSKB
CASA
AKBNK


In [400]:
marketValues.to_pickle("marketValues")